## ETEE Top 40 sentiment with musicmatch

I got tried of trying to figure out which lyrics are positive, so I decided to automate it, by reading this!

https://www.officialcharts.com/chart-news/the-official-top-40-biggest-songs-of-2020-so-far__29264/

Then I got tired and bored with exploring the data by artist and sentiment so I used IPYwidgets interact!

https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html



In [5]:
# Code from the Musixmatch homework!

import requests 
import pandas as pd

azure_key = "your-key"
azure_endpoint = "your-endpoint"
musixmatch_key = 'your-key'


def sentiment (text):
    '''
        returns the sentiment of the given text. up to you as to whether it returns just 'pos', 'neg', 'neutral' or more information 
    '''
    header = { 'Ocp-Apim-Subscription-Key' : azure_key}
    url = f'{azure_endpoint}text/analytics/v3.0/sentiment'
    documents = {"documents": [ {"id": "1", "text": text } ] }

    response = requests.post(url,headers=header, json=documents) # Make Request
    entities = response.json() # de-serialize
    return entities['documents'][0]['sentiment']

def find_song(song_title, song_artist):
    '''
        get the closest matching song to title and artist using the track.search api call
        return the first track
    '''
    url = "http://api.musixmatch.com/ws/1.1/track.search"
    query_string = { 'q_artist' : song_artist,
                    'q_track' : song_title,
                    'apikey' : musixmatch_key }
    response = requests.get(url, params = query_string)
    data = response.json()
    return data['message']['body']['track_list'][0]['track']
    
def get_lyrics(track_id):
    '''
        get the closest matching song to title and artist using the track.lyrics.get api call
        return the lyrics as text string
    '''        
    url = "http://api.musixmatch.com/ws/1.1/track.lyrics.get"
    query_string = { 'track_id' : track_id,
                    'apikey' : musixmatch_key }
    response = requests.get(url, params = query_string)
    #print(response.url)
    data = response.json()
    return data['message']['body']['lyrics']



In [2]:
try:
    title = 'The Gambler'
    artist = 'Kenny Rogerz'
    track = find_song(title, artist)
    lyrics = get_lyrics(track['track_id'])
    l = lyrics['lyrics_body'].replace('******* This Lyrics is NOT for Commercial use *******','')
    sent = sentiment(l)
    print(l)
    print (sent)
except Exception as e:
    print("Cannot find song, artist, or Lyrics from the MusixMatch API?")
    print(f"Actual Error Message: {e}")

Cannot find song, artist, or Lyrics from the MusixMatch API?
Actual Error Message: list index out of range


In [3]:
def sentimizer(title, artist):
    try:
        track = find_song(title, artist)
        lyrics = get_lyrics(track['track_id'])
        l = lyrics['lyrics_body'].replace('******* This Lyrics is NOT for Commercial use *******','')
        sent = sentiment(l)
        return sent
    except Exception as e:
        return 'unknown'

In [6]:
url = 'https://www.officialcharts.com/chart-news/the-official-top-40-biggest-songs-of-2020-so-far__29264/'

tables = pd.read_html(url, header=0)

top40 = tables[0]
top40['SENTIMENT'] = top40.apply( lambda row: sentimizer(row['TITLE'], row['ARTIST']), axis=1 )

top40

,Unnamed: 0,TITLE,ARTIST,PEAK,SENTIMENT
0,1,BLINDING LIGHTS,WEEKND,1,negative
1,2,DANCE MONKEY,TONES & I,1,negative
2,3,ROSES,SAINT JHN,1,negative
3,4,DON'T START NOW,DUA LIPA,2,mixed
4,5,BEFORE YOU GO,LEWIS CAPALDI,1,negative
5,6,ROCKSTAR,DABABY FT RODDY RICCH,1,unknown
6,7,SOMEONE YOU LOVED,LEWIS CAPALDI,1,negative
7,8,OWN IT,STORMZY/ED SHEERAN/BURNA BOY,1,positive
8,9,THE BOX,RODDY RICCH,2,mixed
9,10,SAY SO,DOJA CAT,2,mixed


In [7]:
top40.to_csv('./top40.csv')

In [37]:
sent = input("Enter a sentiment")
top40[ top40['SENTIMENT'] == sent ]

Enter a sentiments


,Unnamed: 0,TITLE,ARTIST,PEAK,SENTIMENT


In [45]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


@interact(sentiment=['any','positive','negative','mixed', 'unknown'], artist='')
def main(sentiment, artist):
    if sentiment == 'any':
        sent40 = top40
    else:
        sent40 = top40[ top40['SENTIMENT'] == sentiment ]
    if artist == "":
        art40 = sent40
    else:
        art40 = sent40[ sent40['ARTIST'].str.find(artist.upper()) >=0 ]
    
    display(art40)

interactive(children=(Dropdown(description='sentiment', options=('any', 'positive', 'negative', 'mixed', 'unkn…

In [47]:
@interact(sentiment=['any','positive','negative','mixed', 'unknown'])
def main(sentiment):
    print(f"this makes me {sentiment}")

interactive(children=(Dropdown(description='sentiment', options=('any', 'positive', 'negative', 'mixed', 'unkn…

In [49]:
@interact(name='', age=(1,100))
def main(name,age):
    next_year= age+1
    print(f"{name} is {age} next year they will be {next_year}")

interactive(children=(Text(value='', description='name'), IntSlider(value=50, description='age', min=1), Outpu…